This following programm organises the data from the inputdataset in a way, that it can be used with the pytred Relevance Evaluator. This programm rankes the fascets randomnly by assisning it the score of the randomn position of the fascet in the list of fascets.

In [1]:
import pandas as pd
import json
import pytrec_eval

In [2]:
dataset_file = '../../Data/Crossencoder/DF_Constructed_CrossEncoder_dataset.csv'

In [3]:
df = pd.read_csv(dataset_file)

In [5]:
df =  df.sort_values('Unnamed: 0')

In [6]:
gk = df.groupby('query')

In [4]:
df

,Unnamed: 0,query,clariQ,fascet,isRelated
0,163793,gmail email address,What do you want to do with gmail email address?,windows 10,0
1,162361,swmp,"Which ""swmp"" do you mean?",storm water management program,1
2,49952,brotherhood of the wolf,What do you want to know about the film?,film,0
3,249336,self less,What do you want to know about the film?,selfless act,0
4,234910,cstc,"Which ""cstc"" do you mean?",cstc pty ltd,0
...,...,...,...,...,...
297147,51810,the pizza place,What do you want to know about this restaurant?,specialty,0
297148,226018,letcher county kentucky,What would you like to know about this place?,zip code,0
297149,139003,ford ka+,What do you want to know about this vehicle?,exterior,1
297150,289701,sterling connecticut,What would you like to know about this city?,zip code,1


In [8]:
qrel = {}

for name, group in gk:
    queryidx = group.iloc[0]['Unnamed: 0']
    el_dict = {}
    for idx, row in group.iterrows():
        if(row['isRelated'] == 1):
            el_dict[row['fascet']] = row['isRelated']
    qrel['q'+str(queryidx)] = el_dict

run = {}
for name, group in gk:
    queryidx = group.iloc[0]['Unnamed: 0']
    el_dict = {}
    group = group.sample(frac=1)
    i = 1
    for idx, row in group.iterrows():
        el_dict[row['fascet']] = (1/i)
        i = i + 1
    run['q'+str(queryidx)] = el_dict


In [9]:
qrel

{'q143608': {'2010': 1, '2013': 1, '2016': 1},
 'q284989': {'javascript': 1, 'python': 1},
 'q194133': {'windows 10': 1, 'windows 8': 1, 'windows 7': 1},
 'q164831': {'windows 10': 1,
  'windows 7': 1,
  'windows vista': 1,
  'server 2012': 1},
 'q228287': {'windows 10': 1,
  'windows 8': 1,
  'windows 7': 1,
  'windows vista': 1},
 'q15689': {'windows 8': 1,
  'windows vista': 1,
  'windows 7': 1,
  'windows xp': 1},
 'q69572': {'download': 1, 'update': 1},
 'q245607': {'windows 10': 1,
  'windows 8': 1,
  'windows 7': 1,
  'windows vista': 1},
 'q24388': {'windows 10': 1,
  'windows 8': 1,
  'windows 7': 1,
  'windows vista': 1},
 'q18185': {'esv': 1, 'kjv': 1, 'niv': 1, 'nlt': 1},
 'q226047': {'esv': 1, 'kjv': 1, 'niv': 1, 'nlt': 1},
 'q70559': {'esv': 1, 'kjv': 1, 'niv': 1},
 'q66588': {'esv': 1, 'kjv': 1, 'niv': 1, 'nlt': 1},
 'q84272': {'esv': 1, 'kjv': 1, 'niv': 1},
 'q259485': {'excel': 1, 'indesign': 1, 'word': 1},
 'q135048': {'tour': 1, 'albums': 1, 'lyrics': 1, 'members': 1

In [10]:
run

{'q143608': {'2013': 0.14285714285714285,
  'sha256': 0.5,
  'windows 10': 0.3333333333333333,
  'windows 8': 0.25,
  '2010': 0.1111111111111111,
  'md5': 0.16666666666666666,
  '32': 0.125,
  'spreadsheetguru': 0.1,
  'mrexcel': 0.09090909090909091,
  'password': 0.08333333333333333,
  '2016': 0.07142857142857142,
  'windows 7': 0.06666666666666667},
 'q284989': {'javascript%2fs': 1.0,
  'html%2f': 0.5,
  'css%3f': 0.3333333333333333,
  'characters': 0.25,
  'python': 0.2,
  'java "%2f': 0.16666666666666666,
  'javascript': 0.14285714285714285,
  'readable format': 0.125,
  'windows 10': 0.1111111111111111,
  'windows 7': 0.1,
  'windows 8': 0.09090909090909091,
  'table': 0.08333333333333333},
 'q194133': {'desktop': 1.0,
  'windows 10': 0.2,
  'mobile': 0.3333333333333333,
  'start menu': 0.25,
  'chrome': 0.16666666666666666,
  'windows 8': 0.125,
  'delete': 0.1111111111111111,
  'windows': 0.1,
  'windows 7': 0.07692307692307693,
  'store': 0.08333333333333333,
  'ui': 0.07142857

In [11]:
evaluator = pytrec_eval.RelevanceEvaluator(
    qrel, {'map', 'ndcg'})


In [13]:
evalResults = evaluator.evaluate(run)

In [14]:
sumMap = 0
sumNdcg = 0
for row in evalResults:
    sumMap = sumMap + evalResults[row]['map']
    sumNdcg = sumNdcg + evalResults[row]['ndcg']
    
    
print("Average Map Score:  " + str(sumMap/len(evalResults)))
print("Average NDCG Score: " + str(sumNdcg/len(evalResults)))

Average Map Score:  0.30015797714920406
Average NDCG Score: 0.5130001780629768


In [15]:
totalScores = {
    "map_sum": sumMap,
    "map_avg": sumMap/len(evalResults),
    "ndcg_sum": sumNdcg,
    "ndcg_avg": sumNdcg/len(evalResults),
}

In [ ]:
with open('Output_CE_dataset_rnd.json', 'w') as f:
    json.dump(evaluator.evaluate(run), f)

with open('Output_CE_dataset_rnd_total.json', 'w') as f:
    json.dump(totalScores, f)